In [1]:
! ls

Digit-Recognizer.ipynb input                  sample_submission.csv


In [2]:
import pandas as pd

In [3]:
sample = pd.read_csv('sample_submission.csv')

In [4]:
sample

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
5,6,0
6,7,0
7,8,0
8,9,0
9,10,0
